### Grabbing the Data from FRED

In [4]:
import pandas as pd                 # Our pandas package
from pandas_datareader import data  # This is will give us access to FRED
import datetime as dt               # This will get us the datatime package
import matplotlib.pyplot as plt     # The new package we are learning about Matplotlib
                                    # pyplot is just one module of many in this library

### Getting the Data and Getting it Ready

Now that we have our packages imported, we want to read in the data. We will use FRED to do so. Recall that we need to provide FRED a start date and then the codes for the data series that we want.


In [5]:
start = dt.datetime(2000,1,1) # Our start date

codes = ["GDPC1", "PCECC96", "GPDIC1", "GCEC1","EXPGSC1","IMPGSC1"] # Our codes

- These codes will give us (quarterly, seasonally adjusted) real gdp, consumption, investment, government expenditures, then net-exports. In other words, we have all the data series that go into the Y = C + I + G + NX identitiy. 
- Here is a [nice intro to FRED doing a similar excercise.](https://www.stlouisfed.org/~/media/Education/Lessons/pdf/FRED-GDP-Stacking.pdf)

In [6]:
usdata = data.DataReader(codes, "fred", start)                 # Read  in the data

usdata.columns = ["gdp", "cons","invs", "gov", "exp", "imp"]   # Rename the columns in a simple way

usdata["nx"] = usdata.exp - usdata.imp                         # Create the net exports column


Just a recap...everything we did above should seem familiar. We read in data using the pandas DataReater. We used our pandas knowledge to rename the columns. Then we created a column by performing an operation on two of the data series.

##### Checking the data. 

Before we go anyfurther, I'm going to do a couple of "sanity checks" of the data. That is, I want to check, look at the data to see if it is consistent with what I'm expecting it to be. In this case, I'm going to look at (i) the consumption share of GDP and (ii) see if (Y- C+I+G+NX)/Y is close to zero (if you are not following along, why does this make sense?). Then my favorite command is to check out the head.

In [7]:
usdata["cons_share"] = usdata.cons / usdata.gdp # Consumption share 
usdata["stat_des"] = (usdata.gdp - usdata.cons - usdata.invs - usdata.gov - usdata.nx)/usdata.gdp # Does GDP match up?
usdata.head() # Look at it

,gdp,cons,invs,gov,exp,imp,nx,cons_share,stat_des
DATE,,,,,,,,,
2000-01-01,12924.179,8520.710,2257.109,2645.393,1337.203,1853.884,-516.681,0.659284,0.001366
2000-04-01,13160.842,8603.007,2390.733,2671.254,1375.203,1911.437,-536.234,0.653682,0.002438
2000-07-01,13178.419,8687.485,2367.274,2659.687,1408.783,1978.137,-569.354,0.659221,0.002529
2000-10-01,13260.506,8762.205,2371.809,2675.695,1396.844,1977.750,-580.906,0.660775,0.002391
2001-01-01,13222.690,8797.280,2264.202,2716.773,1376.171,1946.136,-569.965,0.665317,0.001089


Everything looks good!

Now I'm going to set stuff up so it is ready to plot.

First, I want to express and plot stuff as growth rates, so I'm going to create a new column with the data sereis in percent changes...note below, the percent change is doing quarter to quarter, to annualize the growth rate, multiply it by four...then times it by 100.

Now, to automate things...note what I did: (i) I created a variable list of the things I care about (ii) when through a for loop and created a `new_name` which is simply the varible name with `_growth` added to it then (iii) then compute the annulized percent change and assined it to a new column with the new name.